In [9]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [10]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
#So what we see is movies dataframe consist of all movie info and what genre it have and rating consist of users rating to various movies.

In [14]:
#Now removing the unnecessary column like timestamp from ratings and genres from movies(although genres is a very important feature but it is for content based system where similarity is between movies and contents but right now we need similarity between users)

In [15]:
movies.drop('genres',axis=1,inplace=True)
ratings.drop('timestamp',axis=1,inplace=True)

In [16]:
#Now combining the both the dataframes on movieId

In [18]:
df=movies.merge(ratings,on='movieId')

In [19]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [20]:
#Now we can see for a particular movie many users had given different ratings

In [42]:
new_df=df.dropna(axis=0,subset=['title'])

In [43]:
#now creating a separate dataframe for total number of rating counts

In [44]:
count=(new_df.groupby('title')['rating'].count().reset_index())

In [45]:
count.head()

,title,rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [46]:
count['totalratingcount']=count['rating']
count.drop('rating',inplace=True,axis=1)

In [47]:
count.head()

,title,totalratingcount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [48]:
#Now we have rating count for each and every movie hence we can merge this dataframe to older datafrane

In [50]:
final=new_df.merge(count, left_on = 'title', right_on = 'title', how = 'left')
#Now this final dataframe consist of rating count and total rating count for every movie given by user

In [51]:
final.head()

,movieId,title,userId,rating,totalratingcount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [52]:
rating_threshold=50
final_1=final.query('totalratingcount >=@rating_threshold')
final_1.head()

,movieId,title,userId,rating,totalratingcount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [53]:
final_1.shape

(41362, 5)

In [55]:
features=final_1.pivot_table(values='rating', index='title', columns='userId').fillna(0)
features.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,570,571,572,573,574,575,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,4.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,3.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,4.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [61]:
from scipy.sparse import csr_matrix
final_matrix=csr_matrix(features.values)

In [62]:
from sklearn.neighbors import NearestNeighbors

In [63]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')

In [65]:
model_knn.fit(final_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [98]:
#Now finding the matching movies on the basis of index

print(query_index)
distances, indices = model_knn.kneighbors(features.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)

49


In [99]:
for i in range(0,len(distances.flatten())):
   if i == 0:
        print('Recommendations for {0}:\n'.format(features.index[query_index]))
   else:
        print('{0}: {1}, with distance of {2}:'.format(i, features.index[indices.flatten()[i]], distances.flatten()[i]))



Recommendations for Beetlejuice (1988):

1: Big (1988), with distance of 0.3593956849390836:
2: Edward Scissorhands (1990), with distance of 0.43060896253904957:
3: Austin Powers: International Man of Mystery (1997), with distance of 0.4362342576839707:
4: Ghostbusters (a.k.a. Ghost Busters) (1984), with distance of 0.4404184943634365:


In [100]:
a=features.index


In [101]:
a

Index(['10 Things I Hate About You (1999)', '12 Angry Men (1957)',
       '2001: A Space Odyssey (1968)', '28 Days Later (2002)', '300 (2007)',
       '40-Year-Old Virgin, The (2005)', 'A.I. Artificial Intelligence (2001)',
       'Abyss, The (1989)', 'Ace Ventura: Pet Detective (1994)',
       'Ace Ventura: When Nature Calls (1995)',
       ...
       'Willy Wonka & the Chocolate Factory (1971)',
       'Wizard of Oz, The (1939)', 'Wolf of Wall Street, The (2013)',
       'X-Men (2000)', 'X-Men: The Last Stand (2006)',
       'X2: X-Men United (2003)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Zombieland (2009)', 'Zoolander (2001)'],
      dtype='object', name='title', length=450)

In [111]:
i=0
for i,j in enumerate(a):
  if j=='Goldfinger (1964)':
    print(i)


176


449

Recommendations for Beetlejuice (1988):

1: Big (1988), with distance of 0.3593956849390836:
2: Edward Scissorhands (1990), with distance of 0.43060896253904957:
3: Austin Powers: International Man of Mystery (1997), with distance of 0.4362342576839707:
4: Ghostbusters (a.k.a. Ghost Busters) (1984), with distance of 0.4404184943634365:
5: Indiana Jones and the Temple of Doom (1984), with distance of 0.4422174807172907:
6: Honey, I Shrunk the Kids (1989), with distance of 0.44484680371203733:
7: Breakfast Club, The (1985), with distance of 0.4486749918473675:
8: Who Framed Roger Rabbit? (1988), with distance of 0.4586472262491299:
9: Splash (1984), with distance of 0.46086516424271406:
10: Nightmare Before Christmas, The (1993), with distance of 0.46980341373849965:
11: Ferris Bueller's Day Off (1986), with distance of 0.4706572745510048:
12: Back to the Future Part III (1990), with distance of 0.48574454930760214:
13: Princess Bride, The (1987), with distance of 0.4882298439809456:
14: